In [ ]:
# 1단계: 필요한 라이브러리만 설치
!pip install torch tensorflow numpy

# 2단계: 라이브러리 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import numpy as np

# 3단계: QNet 클래스 정의
class QNet(nn.Module):
    def __init__(self):
        super(QNet, self).__init__()
        self.fc1 = nn.Linear(5, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# 4단계: PyTorch 모델 로드
q = QNet()
q.load_state_dict(torch.load("pong_model.pth"))
q.eval()

# 5단계: PyTorch 가중치 추출
weights = {}
for name, param in q.named_parameters():
    weights[name] = param.detach().cpu().numpy()

print("✅ PyTorch 가중치 추출 완료!")
print(f"레이어 목록: {list(weights.keys())}")

# 6단계: TensorFlow 모델 구축 (동일한 구조)
tf_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(5,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3)
])

# 7단계: 가중치 복사
tf_model.layers[0].set_weights([weights['fc1.weight'].T, weights['fc1.bias']])
tf_model.layers[1].set_weights([weights['fc2.weight'].T, weights['fc2.bias']])
tf_model.layers[2].set_weights([weights['fc3.weight'].T, weights['fc3.bias']])

print("✅ TensorFlow 모델 생성 및 가중치 복사 완료!")

# 8단계: 검증 (PyTorch vs TensorFlow)
test_input = np.random.randn(1, 5).astype(np.float32)

# PyTorch 출력
with torch.no_grad():
    pytorch_output = q(torch.from_numpy(test_input)).numpy()

# TensorFlow 출력
tf_output = tf_model.predict(test_input, verbose=0)

print(f"\n검증:")
print(f"PyTorch 출력:    {pytorch_output}")
print(f"TensorFlow 출력: {tf_output}")
print(f"차이:           {np.abs(pytorch_output - tf_output).max():.6f}")

# 9단계: TFLite로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# 10단계: TFLite 모델 저장
with open("pong_model.tflite", "wb") as f:
    f.write(tflite_model)
print("\n✅ TFLite 변환 완료!")

# 11단계: TFLite 검증
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], test_input)
interpreter.invoke()
tflite_output = interpreter.get_tensor(output_details[0]['index'])

print(f"\nTFLite 출력:     {tflite_output}")
print(f"차이(vs PyTorch): {np.abs(pytorch_output - tflite_output).max():.6f}")

# 12단계: 다운로드
from google.colab import files
files.download('pong_model.tflite')
print("\n✅ 다운로드 완료!")